In [1]:
# !wget https://huggingface.co/datasets/mesolitica/chatgpt-malaysia-hansard-qa/resolve/main/hansard-qa.json

In [2]:
import json
import random

In [3]:
with open('hansard-qa.json') as fopen:
    data = json.load(fopen)

In [4]:
len(data)

42440

In [5]:
data[0]

[{'original': '\n22                                                            DR.30.10.2018                                                                                                                          \n\n \ndan sebagainya. Cukai-cukai yang didapati daripada minuman keras, alkohol dan sebagainya \nmasuk kepada kerajaan, kerajaan bayar gaji kita balik, itu pun haram pada kita.  [Disampuk]  \n\nTimbalan Yang di-Pertua [Dato’ Mohd Rashid Hasnon]: Yang Berhormat Padang \nSerai, sila ajukan soalan. \n\nTuan Karupaiya Mutusami [Padang Serai]: Saya bertanya soalan pada pihak \nkementerian, bolehkah samsu dihapuskan terus, tidak mahu ada samsu dalam negara kita. \n[Dewan tepuk] Terima kasih. \n\nTuan Che Alias bin Hamid [Kemaman]: Sokong, sokong. \n\n■1150 \n\nTuan Chong Chieng Jen: Itu satu isu atau soalan yang begitu serius dan effect-\nnya yang sangat luas. Saya rasa ini perlu dibawa ke dalam Jemaah Menteri untuk \ndipertimbangkan dan diputuskan. Sekian, terima kasih. \n\n \n\n

In [6]:
data_parlimen = []
for d in data:
    try:
        t = d[0]
        a = d[1]
        j_a = json.loads(d[1])
        data_parlimen.append(
            {
                'context': t['original'],
                'qa': j_a
            }
        )
    except:
        pass
    
len(data_parlimen)

42368

In [7]:
data_parlimen[0]

{'context': '\n22                                                            DR.30.10.2018                                                                                                                          \n\n \ndan sebagainya. Cukai-cukai yang didapati daripada minuman keras, alkohol dan sebagainya \nmasuk kepada kerajaan, kerajaan bayar gaji kita balik, itu pun haram pada kita.  [Disampuk]  \n\nTimbalan Yang di-Pertua [Dato’ Mohd Rashid Hasnon]: Yang Berhormat Padang \nSerai, sila ajukan soalan. \n\nTuan Karupaiya Mutusami [Padang Serai]: Saya bertanya soalan pada pihak \nkementerian, bolehkah samsu dihapuskan terus, tidak mahu ada samsu dalam negara kita. \n[Dewan tepuk] Terima kasih. \n\nTuan Che Alias bin Hamid [Kemaman]: Sokong, sokong. \n\n■1150 \n\nTuan Chong Chieng Jen: Itu satu isu atau soalan yang begitu serius dan effect-\nnya yang sangat luas. Saya rasa ini perlu dibawa ke dalam Jemaah Menteri untuk \ndipertimbangkan dan diputuskan. Sekian, terima kasih. \n\n \n\n9.

In [8]:
indices = {i for i in range(len(data_parlimen))}

In [9]:
data_parlimen[0]['qa']['soalan']

[{'soalan': 'Adakah samsu akan dihapuskan terus dari negara kita?',
  'jawapan': 'Ini perlu dibawa ke dalam Jemaah Menteri untuk dipertimbangkan dan diputuskan.'},
 {'soalan': 'Adakah kerajaan bercadang untuk menyemak semula jadual perjalanan bagi kereta api komuter dan ETS dari Gemas ke Kuala Lumpur?',
  'jawapan': 'KTMB pada masa ini masih belum ada perancangan untuk menyemak semula jadual perjalanan bagi perkhidmatan komuter dan ETS daripada Gemas ke Kuala Lumpur.'},
 {'soalan': 'Berapa banyak set ETS yang dimiliki oleh KTMB untuk tujuan operasi?',
  'jawapan': 'KTMB memiliki 15 set train ETS yang aktif untuk tujuan operasi.'}]

In [10]:
from tqdm import tqdm

pairs = []
for i in tqdm(range(len(data_parlimen))):
    query = data_parlimen[i]['context']
    pos = []
    for s in data_parlimen[i]['qa']['soalan']:
        pos.extend(s.values())
    
    neg_i = random.choice(list(indices - {i}))
    neg = []
    for s in data_parlimen[neg_i]['qa']['soalan']:
        neg.extend(s.values())
        
    pairs.append({
        'query': query,
        'pos': pos,
        'neg': neg
    })
    
    for s in data_parlimen[i]['qa']['soalan']:
        pos = [query, s['jawapan']]
        neg_i = random.choice(list(indices - {i}))
        neg = [data_parlimen[neg_i]['context'], data_parlimen[neg_i]['qa']['soalan'][0]['jawapan']]
        
        pairs.append({
            'query': s['soalan'],
            'pos': pos,
            'neg': neg
        })

100%|████████████████████████████████████| 42368/42368 [01:17<00:00, 547.90it/s]


In [11]:
len(pairs)

169482

In [12]:
with open('hansard-qa-pair.jsonl', 'w') as fopen:
    for l in pairs:
        if not len(l['pos']) or not len(l['neg']):
            continue
        fopen.write(f'{json.dumps(l)}\n')